In [ ]:
# ==============================================
# 📦 1️⃣ 安裝依賴與 TensorFlow Models (適用 Python 3.12)
# ==============================================
!apt-get install -qq protobuf-compiler python3-dev python3-pip

# 安裝新版 TensorFlow 與相關套件
!pip install tensorflow==2.19.1 tensorflow_io==0.37.1 matplotlib opencv-python pillow lxml Cython contextlib2 tf_slim

# ==============================================
# 📦 2️⃣ 下載與安裝 TensorFlow Models
# ==============================================
!git clone https://github.com/tensorflow/models.git

# 進入 models/research 資料夾
%cd /content/models/research

# 編譯 .proto 檔案
!protoc object_detection/protos/*.proto --python_out=.

# 安裝 Object Detection API
!cp object_detection/packages/tf2/setup.py .
!python3 -m pip install .


# 回到主目錄
%cd /content


(Reading database ... 125081 files and directories currently installed.)
Preparing to unpack .../python3-pkg-resources_68.1.2-2~jammy3_all.deb ...
Unpacking python3-pkg-resources (68.1.2-2~jammy3) over (59.6.0-1.2ubuntu0.22.04.3) ...
Selecting previously unselected package python3-setuptools.
Preparing to unpack .../python3-setuptools_68.1.2-2~jammy3_all.deb ...
Unpacking python3-setuptools (68.1.2-2~jammy3) ...
Selecting previously unselected package python3-wheel.
Preparing to unpack .../python3-wheel_0.37.1-2ubuntu0.22.04.1_all.deb ...
Unpacking python3-wheel (0.37.1-2ubuntu0.22.04.1) ...
Selecting previously unselected package python3-pip.
Preparing to unpack .../python3-pip_22.0.2+dfsg-1ubuntu0.7_all.deb ...
Unpacking python3-pip (22.0.2+dfsg-1ubuntu0.7) ...
Setting up python3-pkg-resources (68.1.2-2~jammy3) ...
Setting up python3-setuptools (68.1.2-2~jammy3) ...
Setting up python3-wheel (0.37.1-2ubuntu0.22.04.1) ...
Setting up python3-pip (22.0.2+dfsg-1ubuntu0.7) ...
Processing t

In [ ]:
# 建立必要資料夾
import os

os.makedirs("/content/images/train", exist_ok=True)
os.makedirs("/content/images/test", exist_ok=True)
os.makedirs("/content/annotations", exist_ok=True)

print("✅ 資料夾建立完成")

# 下載 xml_to_csv.py 腳本
!wget -q https://raw.githubusercontent.com/datitran/raccoon_dataset/master/xml_to_csv.py -O /content/xml_to_csv.py
print("✅ xml_to_csv.py 已下載")

# 下載 generate_tfrecord.py 腳本
!wget -q https://raw.githubusercontent.com/datitran/raccoon_dataset/master/generate_tfrecord.py -O /content/generate_tfrecord.py
print("✅ generate_tfrecord.py 已下載")

# 建立 label_map.pbtxt
label_map = """item {
  id: 1
  name: 'apple'
}
"""
with open("/content/annotations/label_map.pbtxt", "w") as f:
    f.write(label_map)
print("✅ label_map.pbtxt 已建立")

# 檢查 train/test 中的 .xml 檔案數量
import glob
train_xmls = glob.glob("/content/images/train/*.xml")
test_xmls = glob.glob("/content/images/test/*.xml")

print(f"📦 Train XML 數量: {len(train_xmls)}")
print(f"📦 Test XML 數量: {len(test_xmls)}")

if len(train_xmls) == 0 and len(test_xmls) == 0:
    print("⚠️ 沒有找到任何 .xml 標註檔，請確認檔案格式！")

# 產生 train_labels.csv / test_labels.csv
!python3 /content/xml_to_csv.py -i /content/images/train -o /content/annotations/train_labels.csv
!python3 /content/xml_to_csv.py -i /content/images/test -o /content/annotations/test_labels.csv

# 檢查是否真的產生成功
print("\n📂 /content/annotations 內容：")
!ls -lh /content/annotations


✅ 資料夾建立完成
✅ xml_to_csv.py 已下載
✅ generate_tfrecord.py 已下載
✅ label_map.pbtxt 已建立
📦 Train XML 數量: 0
📦 Test XML 數量: 0
⚠️ 沒有找到任何 .xml 標註檔，請確認檔案格式！
Successfully converted xml to csv.
Successfully converted xml to csv.

📂 /content/annotations 內容：
total 4.0K
-rw-r--r-- 1 root root 33 Nov  5 05:07 label_map.pbtxt


In [ ]:
pbtxt_content = """item {
  id: 1
  name: 'apple'
}
"""

with open("/content/object-detection.pbtxt", "w") as f:
    f.write(pbtxt_content)

print("✅ object-detection.pbtxt 已建立於 /content/annotations/")


✅ object-detection.pbtxt 已建立於 /content/annotations/


In [ ]:
# 下載 xml_to_csv.py
!wget https://raw.githubusercontent.com/datitran/raccoon_dataset/master/xml_to_csv.py -O /content/xml_to_csv.py

# 產生 train_labels.csv
!python3 /content/xml_to_csv.py -i /content/images/train -o /content/annotations/train_labels.csv

# 產生 test_labels.csv
!python3 /content/xml_to_csv.py -i /content/images/test -o /content/annotations/test_labels.csv


--2025-11-05 05:08:03--  https://raw.githubusercontent.com/datitran/raccoon_dataset/master/xml_to_csv.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1089 (1.1K) [text/plain]
Saving to: ‘/content/xml_to_csv.py’

/content/xml_to_csv 100%[===================>]   1.06K  --.-KB/s    in 0s      

2025-11-05 05:08:03 (82.3 MB/s) - ‘/content/xml_to_csv.py’ saved [1089/1089]

Successfully converted xml to csv.
Successfully converted xml to csv.


In [ ]:
!python xml_to_csv.py



Successfully converted xml to csv.


In [ ]:
%%writefile xml_to_csv.py
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (
                root.find('filename').text,
                int(root.find('size/width').text),
                int(root.find('size/height').text),
                member.find('name').text,
                int(member.find('bndbox/xmin').text),
                int(member.find('bndbox/ymin').text),
                int(member.find('bndbox/xmax').text),
                int(member.find('bndbox/ymax').text)
            )
            xml_list.append(value)

    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

def main():
    for folder in ['train', 'test']:
        xml_path = os.path.join('/content/images', folder)
        xml_df = xml_to_csv(xml_path)
        output_csv = f'/content/annotations/{folder}_labels.csv'
        xml_df.to_csv(output_csv, index=False)
        print(f"✅ {output_csv} generated!")

if __name__ == "__main__":
    main()


Overwriting xml_to_csv.py


In [ ]:
!python xml_to_csv.py


✅ /content/annotations/train_labels.csv generated!
✅ /content/annotations/test_labels.csv generated!


In [ ]:
%%writefile /content/generate_tfrecord.py
import os
import io
import pandas as pd
import tensorflow as tf
from PIL import Image
from collections import namedtuple

# 將 class name 轉為 int
def class_text_to_int(row_label):
    if row_label == 'apple':
        return 1
    else:
        return None

# 將 CSV 按 filename 分組
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(filename)) for filename in gb.groups]

# 建立 tf.train.Example
def create_tf_example(group, path):
    img_path = os.path.join(path, group.filename)
    if not os.path.exists(img_path):
        print(f"⚠️ 找不到圖片: {img_path}")
        return None

    with tf.io.gfile.GFile(img_path, 'rb') as fid:
        encoded_jpg = fid.read()
    img = Image.open(io.BytesIO(encoded_jpg))
    width, height = img.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg' if filename.endswith(b'jpg') else b'jpeg'

    xmins, xmaxs, ymins, ymaxs, classes_text, classes = [], [], [], [], [], []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename])),
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename])),
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_jpg])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_format])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmins)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmaxs)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymins)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymaxs)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=classes)),
    }))
    return tf_example

def main():
    for split_type in ['train', 'test']:
        csv_input = f'/content/annotations/{split_type}_labels.csv'
        image_dir = f'/content/images/{split_type}'
        output_path = f'/content/{split_type}.record'

        if not os.path.exists(csv_input):
            print(f"⚠️ CSV 不存在: {csv_input}")
            continue

        examples = pd.read_csv(csv_input)
        if examples.empty:
            print(f"⚠️ CSV 為空: {csv_input}, 無法生成 TFRecord")
            continue

        grouped = split(examples, 'filename')
        with tf.io.TFRecordWriter(output_path) as writer:
            for group in grouped:
                tf_example = create_tf_example(group, image_dir)
                if tf_example:
                    writer.write(tf_example.SerializeToString())
        print(f'✅ TFRecord saved: {output_path}')

if __name__ == '__main__':
    main()


Overwriting /content/generate_tfrecord.py


In [ ]:
!python3 generate_tfrecord.py --csv_input=data/train_labels.csv  --output_path=train.record --image_dir=images/train
!python3 generate_tfrecord.py --csv_input=data/test_labels.csv  --output_path=test.record --image_dir=images/test

2025-11-05 05:08:21.932031: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762319301.952061     807 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762319301.958367     807 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762319301.973686     807 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762319301.973734     807 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762319301.973742     807 computation_placer.cc:177] computation placer alr